### important: whenever you make changes to an imported notebook you need to select "restart the kernel" in this notebook to load the changes then rerun the cells

In [ ]:
# parameters

REFERENCE_MISSION = 2
mission_numbers = [REFERENCE_MISSION, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]

# Alex - anomaly detection
BBOX_COLOR = (255, 0, 0)
DEBUG_IMAGES = False
NUM_BBOX_ITERATIONS = 2

In [ ]:
import import_ipynb
import KyleFunctions as KF
import AlexFunctions as AF
import JoshFunctions as JF
import LoadData as LD
import VisualizeResults as VR

# Load Data

In [ ]:
# load all of the images and regions (bgr, hsv, histogram)
data = LD.run_load_data(mission_numbers, debug_images=False)

# original color images
mission_images_bgr = data[0]
grass_images_bgr = data[1]
building_images_bgr = data[2]

# histogram normalized color images
histogram_mission_images_bgr = data[3]
histogram_grass_images_bgr = data[4]
histogram_building_images_bgr = data[5]

# original hsv images
mission_images_hsv = data[6]
grass_images_hsv = data[7]
building_images_hsv = data[8]

# histogram normalized hsv images
histogram_mission_images_hsv = data[9]
histogram_grass_images_hsv = data[10]
histogram_building_images_hsv = data[11]

# Anomaly Detection

In [ ]:
# Kyle - anomaly detection


# Alex - anomaly detection
anomaly_masks_1 = AF.anomaly_detection_1(mission_numbers, mission_images_hsv, histogram_mission_images_bgr, histogram_mission_images_hsv, num_bbox_iterations=NUM_BBOX_ITERATIONS, bbox_color=BBOX_COLOR, debug_images=True)

# Heatmap

In [ ]:
# Kyle - heatmap
import import_ipynb
import numpy as np
# import KyleFunctions as KF # TODO code needs to be put into function format
import KyleFunctions as KF
import LoadData as LD
import VisualizeResults as VR

heatmap,percents = KF.score_missions(histogram_grass_images_bgr, 5)
print(percents)
AF.plot_images(image_list=heatmap)
for each in percents:
    print(np.sum(each))

# Roof

In [ ]:
# Josh - roof

roof_masks, roof_anomaly_percentages = JF.findRoofAnomalies(histogram_building_images_hsv)
AF.plot_images(roof_masks, ['roof_masks ' + str(mission_number) for mission_number in mission_numbers])
print(roof_anomaly_percentages)

# Visualize Output

In [ ]:
# combine the bounding boxes from anomaly detection
# combine the heatmap and roof mask
# display histograms, plots, metrics, etc

VR.run_visualize_results(mission_numbers, [], histogram_mission_images_bgr, heatmap, percents, roof_masks, roof_anomaly_percentages, debug_images=False)